In [129]:
import re
import pandas as pd
import logging

def check_string_start(s):
    # 匹配以 "# " 开头，后接任意数字（至少一个）的模式
    pattern = r'^# \d+'
    if re.match(pattern, s):
        return 2
    # 匹配以 "# " 开头但无数字的模式
    elif s.startswith("# "):
        return 1
    else:
        return 0
    

def check_table_start_end(s):

    if s.find("<table>")!=-1 and s.find("</table>")!=-1:
        return 1
    elif s.find("<table>")!=-1:
        return 2
    elif s.find("</table>")!=-1:
        return 3
    else:
        return 0


In [ ]:
#file_c='./text/tariff_personal_c_MinerU.md'
def get_chunck(file_name):
    #(1)获取string格式的md文本
    #获取每个header内的内容
    with open(file_name,encoding='utf-8') as f:
        data=f.readlines()
    #(2)将数据处理为["header","context"]的格式
    graph_list=[]
    header=""
    tittle=""
    table_state=False
    for i in range(len(data)):
        line=data[i]
        if line!="\n":
            #查看是否是table
            check_table=check_table_start_end(line)
            #print(check_table)
            if check_table==0 and table_state==False:
                #如果以#开头则说明是header
                check_header=check_string_start(line)
                if check_header==1:
                    header=line.replace("# ","").replace("\n","")
                    tittle=''
                elif check_header==2:
                    tittle=line.replace("# ","").replace("\n","")
                elif check_header==0:
                    text=header+tittle+":"+line.replace("\n","")
                    graph_list.append([header,'text',text])

            elif check_table==1:
                text=header+tittle+line
                graph_list.append([header,'table',text])
            elif check_table==2:
                table_state=True
                table_line=line
            elif check_table==0 and table_state==True:
                table_line+=line
            elif check_table==3:
                table_state=False
                table_line+=line
                text=header+tittle+":"+table_line
                graph_list.append([header,'table',text])
    out_datafrmae=pd.DataFrame(graph_list,columns=["header","data_type",'context'])
    out_datafrmae.to_excel(file_name.replace(".md",'.xlsx').replace("text",'chunks'),index=False)
    logging.info('{0} Success,num:{1}'.format(file_name,len(graph_list)))
            

In [ ]:

file_c='./text/tariff_personal_c_MinerU.md'
get_chunck(file_c)
file_sc='./text/tariff_personal_sc_MinerU.md'
get_chunck(file_sc)
file_e='./text/tariff_personal_e_MinerU.md'
get_chunck(file_e)

INFO:root:./text/tariff_personal_c_MinerU.md Success,num:155
INFO:root:./text/tariff_personal_sc_MinerU.md Success,num:150
INFO:root:./text/tariff_personal_e_MinerU.md Success,num:178
